# Blank Template for Developing

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) { return false; }
// disable widget scrolling

In [ ]:
# Multi-Pass implementation

import xarray as xr

import qnt.ta as qnta
import qnt.backtester as qnbt
import qnt.data as qndata


def load_data(period):
    return qndata.futures.load_data(tail = period, dims = ("time", "field", "asset"))
    # return qndata.cryptofutures.load_data(tail = period, dims = ("time", "field", "asset"))
    # return qndata.stocks.load_data(tail = period, dims = ("time", "field", "asset"))


def strategy(data):
    close = data.sel(field='close')
    ma_slow = close.rolling(time=200).mean().isel(time=-1) # qnta.sma(close, 200).isel(time=-1)
    ma_fast = close.rolling(time=20).mean().isel(time=-1) # qnta.sma(close, 20).isel(time=-1)
    return xr.where(ma_fast > ma_slow, 1, -1)


weights = qnbt.backtest(
    competition_type="futures", # 'cryptofutures', 'stocks'
    load_data=load_data,
    lookback_period=365,
    start_date='2006-01-01',
    strategy=strategy
)


```python
# Single-pass implementation (for prototyping)

import xarray as xr

import qnt.ta as qnta
import qnt.stats as qns
import qnt.data as qndata
import qnt.output as qnout

# load data
data = qndata.futures.load_data(min_date='2005-01-01')

#calc weights
close = data.sel(field='close')
ma_slow = close.rolling(time=200).mean() #  qnta.sma(close, 200)
ma_fast = close.rolling(time=20).mean() # qnta.sma(close, 200)
weights = xr.where(ma_fast > ma_slow, 1, -1)

weights = qnout.clean(weights, data)

#check
qnout.check(weights, data)

# write results
qnout.write(weights)

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice('2006-01-01',None)))
stats.to_pandas().tail()
```